**LICENCE:**
This tutorial contains adaptations of material from [Programming Language Foundations in Agda](https://plfa.github.io/) by Phil Wadler and Wen Kokke. It is licensed under Creative Commons Attribution 4.0 International.

# Natural numbers `ℕ` and inductive definitions

We now explore *inductive types* such as the natural numbers,
which have an infinite number of inhabitants. Functions on inductive types such as `_+_` and `fib` are defined recursively. In Agda, we can write only total functions (though not all of them). This is achieved by imposing severe restrictions on the format of recursive definitions, e.g., by requiring that some argument is strictly decreasing in every recursive call.

````
module code.nat where

data ℕ : Set where
  zero : ℕ
  suc : ℕ → ℕ

-- with the following directive we can use standard numerals 0, 1, ..., as abbreviations for zero, suc zero, ...
{-# BUILTIN NATURAL ℕ #-}

infixl 5 _+_
infixl 6 _*_ -- higher priority than _+_

-- left associative means that "a + b + c" is parsed as "(a + b) + c"

-- recursive functions on an inductive type
_+_ _*_ : ℕ → ℕ → ℕ -- declaration, definitions below

zero + m = m
(suc n) + m = suc (n + m)

zero * _  =  zero
suc m * n  =  n + (m * n)

-- we can ask Agda to compute answers (normalisation) by hitting SHIFT+TAB (twice for better visualisation)
fiftyfive : ℕ
fiftyfive = 11 * 5
````

# Equality
Before proving interesting properties of natural numbers,
we need to be able to state (and prove) that two terms, such as `5 + 3` and `2 * 2 * 2`, have the same normal form.
This is achieved by *types* of the form `x ≡ y`,
where `x y : A` are *values* of (some) type `A`.
We are allowing types to be indexed by values, thus `x ≡ y` is a dependent type.
The type `x ≡ y` is inhabited precisely when `x` and `y` evaluate to the same normal form (say `a`),
and the unique inhabitant thereof is the constructor `refl : a ≡ a`.

We note that `_≡_` is not a primitive in Agda, but it is defined as part of the standard library.
We will not dwell into the details of how `_≡_` is defined, but rather focus on how to use it by means of examples.

````
{-
import Relation.Binary.PropositionalEquality as Eq
open Eq using (_≡_; refl; trans; sym; cong; cong-app; subst) public
open Eq.≡-Reasoning using (begin_; _≡⟨⟩_; _≡⟨_⟩_; _∎) public

-- Agda establishes that `refl : 8 ≡ 8` by normalising the two arguments of `_≡_`;
-- this is one reason why it is fundamental that evaluation always terminates
_ : 5 + 3 ≡ 2 * 2 * 2
_ = refl

-- the following example shows that also terms with free variables are normalised:
-- `0 + m` and `m` evaluate (i.e., normalise) to the same normal form (namely, "m")
_ : {m : ℕ} → 0 + m ≡ m
_ = refl

-- normalisation works also inside contexts:
-- since `0 + m` normalises to `m`, `(0 + m) * n` normalises to `m * n`
_ : {m n : ℕ} → (0 + m) * n ≡ m * n
_ = refl
-}
````

**Exercise (Hello World!).** Define the Fibonacci function `fib` and prove that `fib 26 ≡ 121393`, where
\begin{align*}
    \mathsf{fib}(0) &= 0, \\
    \mathsf{fib}(1) &= 1, \\
    \mathsf{fib}(n) &= \mathsf{fib}(n-1) + \mathsf{fib}(n-2).
\end{align*}
Notice that we didn't define a subtraction operation on natural numbers.

*Hint:* Use pattern matching on the input variable.ell

In [ ]:
module code.fib where
open import code.nat

fib : ℕ → ℕ
fib = ?

## Code

````
module code.eq where

--import Relation.Binary.PropositionalEquality as Eq
--open Eq using (_≡_; refl; trans; sym; cong; cong-app; subst) public
--open Eq.≡-Reasoning using (begin_; _≡⟨⟩_; _≡⟨_⟩_; _∎) public

infix 4 _≡_
data _≡_ {A : Set} (x : A) : A → Set where
  refl : x ≡ x
  
sym : {A : Set} {x y : A} → x ≡ y → y ≡ x
sym refl = refl

trans : {A : Set} {x y z : A} → x ≡ y → y ≡ z → x ≡ z
trans refl refl = refl

cong : {A B : Set} (f : A → B) {x y : A} → x ≡ y → f x ≡ f y
cong f refl = refl

-- equational reasoning
infix  1 begin_
infixr 2 _≡⟨⟩_ _≡⟨_⟩_
infix  3 _∎

begin_ : {A : Set} {x y : A} → x ≡ y → x ≡ y
begin x≡y = x≡y

_≡⟨⟩_ : {A : Set} (x : A) {y : A} → x ≡ y → x ≡ y
x ≡⟨⟩ x≡y = x≡y

_≡⟨_⟩_ : {A : Set} (x : A) {y z : A} → x ≡ y → y ≡ z → x ≡ z
x ≡⟨ x≡y ⟩ y≡z = trans x≡y y≡z

_∎ : {A : Set} (x : A) → x ≡ x
x ∎ = refl

open import code.nat

-- Agda establishes that `refl : 8 ≡ 8` by normalising the two arguments of `_≡_`;
-- this is one reason why it is fundamental that evaluation always terminates
_ : 5 + 3 ≡ 2 * 2 * 2
_ = refl

-- the following example shows that also terms with free variables are normalised:
-- `0 + m` and `m` evaluate (i.e., normalise) to the same normal form (namely, "m")
_ : {m : ℕ} → 0 + m ≡ m
_ = refl

-- normalisation works also inside contexts:
-- since `0 + m` normalises to `m`, `(0 + m) * n` normalises to `m * n`
_ : {m n : ℕ} → (0 + m) * n ≡ m * n
_ = refl
````

# Inductive proofs

````
module code.nat.ind where
open import code.nat public
open import code.eq public
````

We have seen how the set of natural numbers can be defined inductively,
and we have also seen how some identities on natural numbers such as `0 + m ≡ m`
can be proved by `refl : m ≡ m` because `0 + m` normalises to `m`.

Consider now the similarly looking property

````
m+0≡m : (m : ℕ) → m + 0 ≡ m
````

(Notice that `m+0≡m` is a valid identifier, and it is the name of our theorem.)
Now `refl` won't cut it, because`m + 0` is already in normal form and syntactically different from `m`.
In order to prove `m+0≡m` we need induction.

In Agda, a proof by induction is a recursive program
which transforms a proof for case $n$ into a proof for case $n+1$. 

The function `_+_` has two parameters, so we need to choose which one to do induction on.
Since `_+_` is defined by recursion on its first argument,
it is good heuristics to do induction on the first argument.

We now proceed with the proof, i.e., with the recursive definition of `m+0≡m`.
In the base case, we have `m = 0` and thus we need to prove `0 + 0 ≡ 0`,
which follows from reflexivity:

````
m+0≡m 0 = refl
````

In the inductive step, we need to prove `suc m + 0 ≡ suc m` (by defining `m+0≡m (suc m) : suc m + 0 ≡ suc m`)
under the assumption that `m + 0 ≡ m` holds (as witnessed by `m+0≡m m : m + 0 ≡ m`).
The term `suc m + 0` rewrites to `suc (m + 0)`, according to the definition of `_+_`.
Thus, it suffices to show `suc (m + 0) ≡ suc m`.
By inductive assumption, `m + 0 ≡ m` holds, and it suffices to lift the equivalence to the context `suc (...)`.
This is what the program `cong` (for *congruence*) does:

````
m+0≡m (suc m) = cong suc (m+0≡m m)
````

## Symmetry

We now have our first proof by induction `m+0≡m : (m : ℕ) → m + 0 ≡ m`.
Suppose we now want to prove the symmetric statement, i.e.,

````
m≡m+0 : (m : ℕ) → m ≡ m + 0
````

One way would be to reason by induction on `m`, and essentially repeate the proof for `m+0≡m`.
A better way is to use the principle *symmetry* applied to the previous `m+0≡m`,
which is what the expression `sym : ... → x ≡ y → y ≡ x` does below.

````
m≡m+0 m = sym (m+0≡m m)
````

## Equality chains
We can make proofs using chains of equalities more readable by using some syntactic sugaring.
This also helps documenting the proof, i.e., showing what is the *intention* behind the code.
In order to illustrate the use of equality chains, we repeat the proofs above in this new style.

````
m+0≡m' : (m : ℕ) → m + 0 ≡ m
m+0≡m' 0 = refl
m+0≡m' (suc m) =
    suc m + 0 ≡⟨⟩ -- by the definition of _+_
    suc (m + 0) ≡⟨ cong suc (m+0≡m' m) ⟩ -- inductive hypothesis
    suc m ∎
  
m≡m+0' : (m : ℕ) → m ≡ m + 0
m≡m+0' m =
    m ≡⟨ sym (m+0≡m' m) ⟩
    m + 0 ∎
````

### Associativity of  `_+_`

````
+-assoc : (m n l : ℕ) → m + (n + l) ≡ m + n + l
+-assoc 0 n l =
    0 + (n + l)
    ≡⟨⟩ n + l
    ≡⟨⟩ (0 + n) + l ∎
+-assoc (suc m) n l =  
    suc m + (n + l) ≡⟨⟩
    suc (m + (n + l)) ≡⟨ cong suc (+-assoc m n l) ⟩ -- inductive hypothesis
    suc ((m + n) + l) ≡⟨⟩
    suc (m + n) + l ≡⟨⟩
    (suc m + n) + l ∎
````

**Exercise.** In this exercise we prove that addition is commutative. We proceed in two steps.

1. Prove the following auxiliary fact about `_+_` and `suc`:

```agda
suc-lemma : (m n : ℕ) → suc (m + n) ≡ m + suc n
```
*Hint:* Use chains of equations to keep track of normal forms.

2. Prove that addition is commutative:

```agda
+-comm : (m n : ℕ) → m + n ≡ n + m
```

*Hint:* Use `m≡m+0` from above, and `suc-lemma`.

*General hint:* When there are several variables to do induction on, a good heuristics is to do induction on the variable(s) that appear as recursive arguments. For instance, if we have an expression `x + y`, it is probably more covenient to reason by induction on `x`, since `_+_` is defined by induction on its first argument.

In [ ]:
module code.nat.comm where
open import code.nat.ind

suc-lemma : (m n : ℕ) → suc (m + n) ≡ m + suc n
suc-lemma = ?

+-comm : (m n : ℕ) → m + n ≡ n + m
+-comm = ?

# Relations
We explore how inductive definitions can be used to define some common unary and binary relations on the natural numbers.

## Equality on `N`

### **TODO**

Figure out whether to present equality for `ℕ`
or general equality first.

````
module code.nat.eq where
open import code.nat

infix 4 _≡ℕ_

-- indexed inductive type
data _≡ℕ_ : ℕ → ℕ → Set where
  z≡z : 0 ≡ℕ 0
  s≡s : {m n : ℕ} → m ≡ℕ n → suc m ≡ℕ suc n

-- some examples
_ : 1 ≡ℕ 1
_ = s≡s {0} {0} z≡z  -- could omit the implicit arguments

_ : 4 ≡ℕ 4
_ = s≡s (s≡s (s≡s (s≡s z≡z)))

≡ℕ-refl : {n : ℕ} → n ≡ℕ n
≡ℕ-refl {0} = z≡z
≡ℕ-refl {suc n} = s≡s (≡ℕ-refl {n}) -- we could omit {n}, left for clarity here

≡ℕ-trans : {m n p : ℕ} → m ≡ℕ n → n ≡ℕ p → m ≡ℕ p
≡ℕ-trans {0} {0} {0} _ _ = z≡z
≡ℕ-trans {suc m} {suc n} {suc p} (s≡s m≡n) (s≡s n≡p) = s≡s (≡ℕ-trans m≡n n≡p)

≡ℕ-sym : {m n : ℕ} → m ≡ℕ n → n ≡ℕ m
≡ℕ-sym {0} {0} _ = z≡z
≡ℕ-sym {suc m} {suc n} (s≡s m≡n) = s≡s (≡ℕ-sym {m} {n} m≡n)

postulate ≡ℕ-cong : {m n : ℕ} (f : ℕ → ℕ) → m ≡ℕ n → f m ≡ℕ f n

-- infix  1 begin_
infixr 2 _≡ℕ⟨⟩_ _≡ℕ⟨_⟩_
infix  3 _∎ℕ

_≡ℕ⟨⟩_ : (x : ℕ) {y : ℕ} → x ≡ℕ y → x ≡ℕ y
x ≡ℕ⟨⟩ x≡y = x≡y

_≡ℕ⟨_⟩_ : (x : ℕ) {y z : ℕ} → x ≡ℕ y → y ≡ℕ z → x ≡ℕ z
x ≡ℕ⟨ x≡y ⟩ y≡z = ≡ℕ-trans x≡y y≡z

_∎ℕ : (x : ℕ) → x ≡ℕ x
x ∎ℕ = ≡ℕ-refl

0+m≡m : (m : ℕ) → 0 + m ≡ℕ m
0+m≡m m =
    0 + m ≡ℕ⟨⟩
    m ≡ℕ⟨ ≡ℕ-refl {m} ⟩
    m ∎ℕ

m+0≡m : (m : ℕ) → m + 0 ≡ℕ m
m+0≡m 0 = 
    0 + 0 ≡ℕ⟨⟩
    0 ≡ℕ⟨ ≡ℕ-refl ⟩
    0 ∎ℕ
m+0≡m (suc m) =
    suc m + 0 ≡ℕ⟨⟩
    suc (m + 0) ≡ℕ⟨ s≡s (m+0≡m m) ⟩
    suc m ∎ℕ

m≡m+0 : (m : ℕ) → m ≡ℕ m + 0
m≡m+0 m =
    m ≡ℕ⟨ ≡ℕ-sym (m+0≡m m) ⟩
    m + 0 ∎ℕ
    
+-assoc : (m n l : ℕ) → m + (n + l) ≡ℕ m + n + l
+-assoc 0 n l =
    0 + (n + l) ≡ℕ⟨⟩
    n + l ≡ℕ⟨⟩
    (0 + n) + l ∎ℕ
+-assoc (suc m) n l =  
    suc m + (n + l) ≡ℕ⟨⟩
    suc (m + (n + l)) ≡ℕ⟨ s≡s (+-assoc m n l) ⟩ -- inductive hypothesis
    suc ((m + n) + l) ≡ℕ⟨⟩
    suc (m + n) + l ≡ℕ⟨⟩
    (suc m + n) + l ∎ℕ
    
suc-lemma : (m n : ℕ) → suc (m + n) ≡ℕ m + suc n
suc-lemma 0 n =
    suc (0 + n) ≡ℕ⟨⟩
    suc n ≡ℕ⟨⟩
    0 + suc n ∎ℕ
suc-lemma (suc m) n = 
    suc (suc m + n) ≡ℕ⟨⟩
    suc (suc (m + n)) ≡ℕ⟨ s≡s (suc-lemma m n) ⟩ 
    suc (m + suc n) ≡ℕ⟨⟩
    suc m + suc n ∎ℕ

+-comm : (m n : ℕ) → m + n ≡ℕ n + m
+-comm 0 n =
    0 + n ≡ℕ⟨⟩
    n ≡ℕ⟨ m≡m+0 n ⟩
    n + 0 ∎ℕ
+-comm (suc m) n = 
    suc m + n ≡ℕ⟨⟩
    suc (m + n) ≡ℕ⟨ s≡s (+-comm m n) ⟩
    suc (n + m) ≡ℕ⟨ suc-lemma n m ⟩
    n + suc m ∎ℕ
````

## Order

````
module code.nat.leq where
open import code.nat public
open import code.eq public
````

In the same way as we define the natural number inductively,
we can also define relations on the natural numbers inductively.
For instance, we can define the total order relation between natural numbers `m ≤ n`,
which is inhabited precisely when `m` is less or equaly than `n` in the natural order:

````
infix 4 _≤_
data _≤_ : ℕ → ℕ → Set where
  0≤n : {n : ℕ} → 0 ≤ n
  s≤s : {m n : ℕ} → m ≤ n → suc m ≤ suc n
````

In the first case, we say that `0 ≤ n` always holds,
as witnessed by the (base) constructor `0≤n`.
In the second case, if `m ≤ n` holds (as witnessed by some `m≤n : m ≤ n`),
then `suc m ≤ suc n` also holds.
The witness of the later fact is given by the second (inductive) constructor `s≤s m≤n : suc m ≤ suc n`
applied to `m≤n`.

The type `m ≤ n` is called an *indexed type*
since it depends on the indices `m` and `n`,
which vary in the definition of `_≤_`.

````
-- some examples
_ : 1 ≤ 100
_ = s≤s {0} {99} (0≤n {99}) -- could omit the implicit arguments

_ : 4 ≤ 1000
_ = s≤s (s≤s (s≤s (s≤s 0≤n)))
````

### A characterisation

````
module code.nat.leq.char where
open import code.nat.leq public
open import code.universal public
open import code.existential public
````

As an application of existential quantification,
we show that `m ≤ n` implies that there is some `p` s.t. `m + p ≡ n`:

````
q : ∀[ m ] ∀[ n ] (m ≤ n → ∃[ p ] m + p ≡ n)
````

The proof is by induction on `m` (which is also simultaneously an induction on `m ≤ n`).
In the base case, `m ≡ 0` and we can just take as witness for `p` the value `n`:

````
q zero n _ = ⟨ n , refl ⟩
````

In the inductive case, we have

````
q (suc m) (suc n) (s≤s m≤n) with q m n m≤n
... | ⟨ p , m+p≡n ⟩ = ⟨ p , cong suc m+p≡n ⟩
````

We recursively call `q m n m≤n` to collect a witness `⟨ p , m+p≡n ⟩ : ∃[ p ] m + p ≡ n`,
where `p : ℕ` and `m+p≡n : m + p ≡ n`.
Then, `⟨ p , cong suc m+p≡n ⟩ : ∃[ p ] suc m + p ≡ suc n`
because `suc m + p` normalises to `suc (m + p)`.

We do not have to worry about the other case `q (suc m) zero (s≤s m≤n)`
because Agda correctly detects that it cannot possibly occurr as witnessed by `s≤s m≤n`.

This is another yet example of external verification.

### **Exercise**

Show the other direction, namely,

```agda
r : ∀[ m ] ∀[ n ] (∃[ p ] m + p ≡ n → m ≤ n)
```
Proceed by induction on `m`.
In the inductive case, use the fact that `suc` is injective (`suc-injective` below) to derive `m + p ≡ n`,
which can be used to call `r` recursively.

````
suc-injective : ∀[ m ] ∀[ n ] (suc m ≡ suc n → m ≡ n)
suc-injective m n refl = refl
````

In [ ]:
module code.nat.leq-exists2 where
open import code.nat.leq.char public

r : ∀[ m ] ∀[ n ] (∃[ p ] m + p ≡ n → m ≤ n)
r zero _ _ = ?
r (suc m) (suc n) ⟨ p , sucm+p≡sucn ⟩ = ?
    where
        m+p≡n : m + p ≡ n
        m+p≡n = ?
        m≤n : m ≤ n
        m≤n = ?

In [ ]:
module code.nat.leq-exists2 where
open import code.nat.leq.char public

r : ∀[ m ] ∀[ n ] (∃[ p ] m + p ≡ n → m ≤ n)
r zero _ _ = 0≤n
r (suc m) (suc n) ⟨ p , sucm+p≡sucn ⟩ = s≤s m≤n
    where
        m+p≡n : m + p ≡ n
        m+p≡n = suc-injective (m + p) n sucm+p≡sucn
        m≤n : m ≤ n
        m≤n = r m n ⟨ p , m+p≡n ⟩

## Properties of relations

````
module code.nat.leq.prop where
open import code.nat.leq public
````

We formalise and prove that `_≤_` satisfies some common relational properties.

### Reflexivity

We begin by giving an example proof that `_≤_` is *reflexive*, in the sense that

````
≤-refl : {n : ℕ} → n ≤ n
````

(We leave the parameter implicit because 1) Agda can infer it most of the times,
and 2) it will make the following code more readable.)
The proof that `_≤_` is reflexive is by induction:

````
≤-refl {0}     = 0≤n
≤-refl {suc n} = s≤s (≤-refl {n})
````

In the base case, we just use the fact that `0≤n {0} : 0 ≤ 0`.
In the inductive case, the inductive hypothesis `≤-refl {n} : n ≤ n` is a proof that `n ≤ n` holds,
and thus `s≤s (≤-refl {n}) : suc n ≤ suc n`, as required.

### Transitivity

We show that `_≤_` is *transitive*, in the sense that

````
≤-trans : {m n p : ℕ} → m ≤ n → n ≤ p → m ≤ p
````

While the previous proof was by induction on the natural number argument `{n : ℕ}`,
for transitivity we do induction on the *evidence* that `m ≤ n` holds.

````
≤-trans 0≤n _               = 0≤n
≤-trans (s≤s m≤n) (s≤s n≤p) = s≤s (≤-trans m≤n n≤p)
````

In the base case, the first (non-implicit) argument is `0≤n : 0 ≤ n`, from which we can deduce `m ≡ 0`,
and consequently `0≤n : 0 ≤ p` is the term we are after; notice that the two occurrences of `0≤n` have different types (implicit parameters hide this information).
In the inductive case, the first argument is `s≤s m≤n : suc m ≤ suc n`
and the second argument is `s≤s n≤p : suc n ≤ suc p`,
where `m≤n : m ≤ n` and `n≤p : n ≤ p`.
By inductive assumption, `≤-trans m≤n n≤p : m ≤ p`,
and thus `s≤s (≤-trans m≤n n≤p) : suc m ≤ suc p`, as required.
Notice that the case

````
-- ≤-trans (s≤s m≤n) 0≤n
````
cannot arise: The shape of the first argument `s≤s m≤n` implies that `n` is of the form `suc _`,
and thus it cannot be `0`, as implied by the second argument.
Agda detects that this case cannot occurr
and does not complain that the corresponding definition is missing.

**Exercise.** Show that addition by a natural number is an nondecreasing function:

```agda
≤+ : {m n : ℕ} → m ≤ m + n
```

*Hint:* Do induction on the inductive argument of `_+_`.

In [ ]:
module code.nat.leq.plus where
open import code.nat.leq

≤+ : {m n : ℕ} → m ≤ m + n
≤+ {m} {n} = ?

### Antisymmetry

**Exercise**. Prove that `_≤_` is *anti-symmetric*, in the sense that

```agda
≤-asym : {m n : ℕ} → m ≤ n → n ≤ m → m ≡ n
```

*Hint:* Do induction on `_≤_`.
As in transitivity, in the inductive step not all cases can arise (two out of four).

In [ ]:
module code.nat.leq.asym where
open import code.nat.leq

≤-asym : {m n : ℕ} → m ≤ n → n ≤ m → m ≡ n
≤-asym = ?

### Totality

**Exercise.** Prove that `_≤_` is total, in the sense below.

In [ ]:
module code.nat.leq.total where
open import code.nat.leq
open import code.universal
open import code.or public

≤-total : ∀[ m ] ∀[ n ] m ≤ n ∨ n ≤ m
≤-total = ?

In [ ]:
module code.nat.leq.total where
open import code.nat.leq
open import code.universal
open import code.or public

≤-total : ∀[ m ] ∀[ n ] m ≤ n ∨ n ≤ m
≤-total zero _ = left 0≤n
≤-total _ zero = right 0≤n
≤-total (suc m) (suc n) with ≤-total m n
... | left m≤n = left (s≤s m≤n)
... | right n≤m = right (s≤s n≤m)

# Mutually inductive definitions

**Exercise.** Define the unary relations (predicates) on the natural numbers `Even n` and `Odd n`
by a mutual inductive definition.
This is possible by separating the *declaration* of `Even` and `Odd` from their *definition*, as shown below.

In [ ]:
module code.nat.even-odd where
open import code.nat public

data Even : ℕ → Set
data Odd : ℕ → Set

data Even where
    zero : ?
    suc : ?
    
data Odd where
    suc : ?
    
-- note that we are overloading the constructor zero (twice, since also zero : ℕ) and suc (thrice!);
-- this is allowed since the typing context always disambiguates the constructor

# External verification

**Exercise.** Show that
- the sum of two even numbers is an even number,
- the sum of an odd and an even number is an odd number (2 cases), and
- the sum of two odd numbers is an even number.

*Hint:* It's convenient to define four mutually recursive functions.

This is an example of *external verification*, whereby we are proving a property of the sum function `_+_` separately (externally) from its definition.
We will see later on an example of *internal verification*,
whereby the property is hard-wired inside the function.

In [ ]:
module code.nat.even-odd.sum where
open import code.nat.even-odd public

-- declarations
plusEE : {m n : ℕ} → Even m → Even n → Even (m + n)
plusEO : {m n : ℕ} → Even m → Odd n → Odd (m + n)
plusOE : {m n : ℕ} → Odd m → Even n → Odd (m + n)
plusOO : {m n : ℕ} → Odd m → Odd n → Even (m + n)

-- definitions
plusEE = ?
plusEO = ?
plusOE = ?
plusOO = ?

# Challenges

In this last part we propose some more advanced exercises.
No additional tools are required, but the solutions are more elaborated.

## Distributivity for `_+_` and `_*_`

**Exercise**. In this exercise we prove properties relating addition and multiplication.
1. Prove that right multiplication distributes over addition,
in the sense that
```agda
+*-distr : (a b c : ℕ) → (a + b) * c ≡ a * c + b * c
```
2. Prove that multiplication is associative. *Hint: use* `+*-distr`.
3. Prove that multiplication is commutative.
It might be convenient to preliminarily prove the first two properties:
```agda
*-zero : (a : ℕ) → a * 0 ≡ 0
*-suc : (a b : ℕ) → a * suc b ≡ a + a * b
*-comm : (a b : ℕ) → a * b ≡ b * a
```
4. Prove that left multiplication distributes over addition, in the sense that
```agda
*+-distr : (a b c : ℕ) → a * (b + c) ≡ a * b + a * c
```
*Hint: use the previous points*.
5. Using the results above, prove the following
```agda
*+-full : (a b c d : ℕ) → (a + b) * (c + d) ≡ a * c + a * d + b * c + b * d
```
*Advice*: If an induction is required, it's convenient to proceed on the first argument `a : ℕ`.

In [ ]:
module code.nat.prop where
open import code.nat
open import code.nat.assoc
open import code.nat.comm

-- 1.
+*-distr : (a b c : ℕ) → (a + b) * c ≡ a * c + b * c
+*-distr = ?

-- 2.
*-assoc : (a b c : ℕ) → a * b * c ≡ a * (b * c)
*-assoc = ?

-- 3.
*-zero : (a : ℕ) → a * 0 ≡ 0
*-zero = ?

*-suc : (a b : ℕ) → a * suc b ≡ a + a * b
*-suc = ?

*-comm : (a b : ℕ) → a * b ≡ b * a
*-comm = ?

-- 4.
*+-distr : (a b c : ℕ) → a * (b + c) ≡ a * b + a * c
*+-distr = ?

## Monotonicity of `_*_`

**Exercise.** Show that multiplication by a strictly positive number is an increasing function:

```agda
≤* : {m n : ℕ} → 1 ≤ n → m ≤ m * n
```

We proceed in three steps:

1. Show that `_≤_` is preserved if we replace the r.h.s. modulo `_≡_`:

```agda
≤-≡-right : {m n p : ℕ} → n ≡ p → m ≤ n → m ≤ p
```

*Hint:* Use induction on the evidence that `m ≤ n` holds.

2. Show

```agda
≤+2 : {m n : ℕ} → m ≤ n + m
```

This is very similar to `≤+ : {m n : ℕ} → m ≤ m + n` that we proved in module `code.nat.leq.plus`.
*Hint:* Use `≤+`, `≤-≡-right`, and commutativity of addition `+-comm`.

3. Finally, show `≤*`.

*Hint:* Do induction on the inductive argument of `_*_`.
In the inductive case, apply `≤+2` and transitivity `≤-trans`.

In [ ]:
module code.nat.leq.mul where
open import code.nat.leq.plus
open import code.nat.comm
open import code.eq

≤-≡-right : {m n p : ℕ} → n ≡ p → m ≤ n → m ≤ p
≤-≡-right refl a≤b = ?

≤+2 : {m n : ℕ} → m ≤ n + m
≤+2 {m} {n} = ?

≤* : {m n : ℕ} → 1 ≤ n → m ≤ m * n
≤* {m} {n} 1≤n = ?

In [ ]:
module code.nat.leq.mul where
open import code.nat.comm
open import code.nat.leq.plus
open import code.eq

≤-≡-right : {a b c : ℕ} → b ≡ c → a ≤ b → a ≤ c
≤-≡-right refl 0≤n = 0≤n
≤-≡-right refl (s≤s p) = s≤s p

≤+2 : {m n : ℕ} → m ≤ n + m
≤+2 {m} {n} = ≤-≡-right (+-comm m n) (≤+ {m} {n})

≤* : {m n : ℕ} → 1 ≤ n → m ≤ m * n
≤* {zero} {_} _ = 0≤n
≤* {suc m} {suc n} 1≤n = s≤s (≤-trans (≤* {m} {suc n} 1≤n) (≤+2 {m * suc n} {n}))

## Even numbers and divisibility

````
module code.nat.even-odd-div2 where
open import code.universal
open import code.existential hiding (q)
open import code.eq
open import code.nat
open import code.nat.even-odd
````

We want to show that even numbers are divisible by two.
In fact, it is easier to provide such a characterisation simultaneously for even and odd numbers:

````
p : ∀[ n ] (Even n → ∃[ m ] n ≡ m * 2)
q : ∀[ n ] (Odd n → ∃[ m ] n ≡ 1 + m * 2)
````

The code for `p` goes as follows

````
p _ zero = ⟨ zero , refl ⟩
p (suc n) (suc odd) with q n odd
... | ⟨ m , refl ⟩ = ⟨ suc m , refl ⟩
````

In the inductive case, we call recursively `q n odd : ∃[ m ] n ≡ 1 + m * 2`
and we pattern match `⟨ m , refl ⟩` against its result.
Thus, `m : ℕ` and, more importantly, `refl : n ≡ 1 + m * 2`
tells us that `n` is the same as the normal form for `1 + m * 2`,
namely `suc (m * 2)`.
Consequently, `suc n ≡ suc m * 2`,
since `suc m * 2` goes in one step to `2 + m * 2`,
which normalises (in two steps) to `suc (suc (m * 2))`.
Therefore, we return `⟨ suc m , refl ⟩`.

**Exercise.** Complete the code with the definition for `q`.

````
q = ?
````